# Biosound data - Tidy up 
Prep the biosound data for use in the dashboard

To run this in Pycharm you need to set up a kernel that points to the venv:
`python -m ipykernel install --user --name=venv-osa`

Also please note that the data files and folders in this notebook point to a specific path a local Google Drive shared folder. This will not work for others - the paths will need updating for each person.

In [1]:
import pandas as pd
import numpy as np

import data_wrangler as dw
import os

In [2]:
# Set up some initial things
# Local path to the main "Processed Data" folder
local_data_folder = ('/Users/michelle/Library/CloudStorage/GoogleDrive-michelle@waveformanalytics.com/.shortcut'
                     '-targets-by-id/1QNPk7Z3Yb2uhHsHCf49pw7k_2-ivhIJg/Processed_Data')

# config_file = os.path.join(local_data_folder, 'BioSound_Datasets_MapApp.csv')
config_file = os.path.join(local_data_folder, 'BioSound_Datasets_MapApp_v1.5.csv')
df_config = pd.read_csv(config_file)

INDEX_DATA_FOLDER = os.path.join(local_data_folder, 'Revised_Indices_AllData_v2')
# INDEX_DATA_FOLDER = os.path.join(local_data_folder, 'Original_Indices_AllData_v1')

# Folder containing annotations data files
annotations_folder = os.path.join(local_data_folder, 'Annotations')

# Load the fish/annotations codes lookup table
df_codes = pd.read_csv("../shiny/data/fish_codes.csv")

# Satellite water class data
seascaper_folder = os.path.join(local_data_folder,"All_SeascapeR")

# Output file path for the duckdb file 
db_file = os.path.join(local_data_folder, "mbon10.duckdb")

In [3]:
# Take a look at the contents of the config file
df_config

,Dataset,short name,Source,Contact,Sampling Rate (kHz),Bit Rate,tz in file,tz local,Recording Cycle,Hydrophone Latitude,Hydrophone Longitude,Instrument Depth (m),Ecosystem Type,Selected Time Periods,Seascaper File,Annotations File
0,"Biscayne Bay, FL","Biscayne Bay, FL",University of Miami,Neil Hammerschlag/Abby Tinari,32,16,America/New_York,America/New_York,10 seconds every minute,25.396,-80.237,NaN,Mangrove,Feb - Mar 2019,CaesarCreek_SeascapeR_20190101_to_20190401.csv,NaN
1,"Chukchi Sea, Hanna Shoal","Chukchi Sea, Hanna Shoal",Oregon State University,Kate Stafford,16,16,UTC,America/Nome,25 minutes every hour,71.600,-161.500,30.0,Offshore,Jan - Feb 2019,ChuckChi_Sea_SeascapeR_20190101_to_20190401.csv,Chuckchi_Sea_Vessel_2019_02_ISO.csv
2,"Gray's Reef, GA","Gray's Reef, GA",NEFSC-SanctSound,Tim Rowell,48,16,UTC,America/New_York,Continuous five hour files,31.396,-80.890,16.0,Offshore,Jan - Feb 2019,GraysReef_SeascapeR_20190101_to_20190401.csv,sanctsound_products_detections_gr01_sanctsound...
3,"Key West, FL","Key West, FL",Florida Fish & Wildlife,Jess Keller,48,16,America/New_York,America/New_York,30 seconds every five minutes,24.442,-81.934,23.0,Coral Reef,Feb - Mar 2020,KeyWest_SeascapeR_20200101_to_20200401.csv,KW_Annotations.csv
4,"May River, SC","May River, SC",University of S. Carolina Beaufort,Alyssa Marian/Eric Montie,80,16,America/New_York,America/New_York,2 minutes every hour,32.195,-80.792,4.5,Estuary,Jan - Mar 2019,MayRiver_SeascapeR_20190101_to_20190401.csv,MR_Annotations.csv
5,"Olowalu (Maui, HI)","Olowalu (Maui, HI)",SanctSound - Hawaiian Islands Humpback NMS,Eden Zang,48,16,UTC,Pacific/Honolulu,Continuous five hour files,20.807,-156.655,59.7,Island/ Nearshore,Jan - Feb 2019,HI01_NMS_SeascapeR_20190101_to_20190401.csv,sanctsound_products_detections_hi01_sanctsound...
6,ONC-MEF,ONC-MEF,Ocean Networks Canada,Jasper Kanes,64,24,UTC,America/Los_Angeles,Continuous 5 minute files,47.949,-129.098,2189.0,Offshore,Jan - Feb 2019,ONC_MEF_SeascapeR_20190101_to_20190401.csv,ONC_MEF_Vessels_2019_02_ISO.csv
7,OOI-HYDBBA106,OOI-HYDBBA106,Ocean Observatories Initiative,Liz Ferguson,64,16,UTC,America/Los_Angeles,Continuous 5 minute files,44.637,-124.306,80.0,Oregon Shelf,Jan - Feb 2019,HYDBBA106_SeascapeR_20190101_to_20190401.csv,OOI_HYDBBA105_Vessel_2019_02_ISO.csv


## Annotations data

### Prep Key West annotations
Key west annotations are a bit different from the others. They're in a folder that contains several .txt files that need to first be merged and then copied into the same annotations folder as the other datasets.

In [4]:
KW_ANNOTATIONS_FOLDER = os.path.join(local_data_folder, 'Annotations/key-west-original')
KW_ANNOTATIONS_OUTFILE = os.path.join(local_data_folder, 'Annotations/KW_Annotations.csv')

# Run the function from data_wrangler to convert to the regular annotations format and move the file to the same location as the rest of the annotation files
df_kw_anno = dw.annotation_prep_kw_style(KW_ANNOTATIONS_FOLDER, KW_ANNOTATIONS_OUTFILE)

### Prep May River annotations data

May River annotations are also unique. They're stored in wide format in the "Master_Manual....xlsx" file. In this section we re-format so that it looks more like the Key West annotations and all the ship annotations (long-format).

In [5]:
MR_ANNOTATIONS_FILE = os.path.join(local_data_folder, 'Annotations/may-river-original/Master_Manual_14M_2h_011119_071619.xlsx')
MR_ANNOTATIONS_OUTFILE = os.path.join(local_data_folder, 'Annotations/MR_Annotations.csv')

df_mr_anno = dw.annotation_prep_mr_style(MR_ANNOTATIONS_FILE, MR_ANNOTATIONS_OUTFILE, df_codes)

## Index data
Prepare and load the data from the various acoustic index files and combine them all into a big dataframe. Also create a second dataframe that has all the same index values but normalized. 

In [6]:
df_aco0 = dw.prep_index_data(INDEX_DATA_FOLDER, normalize=False)
df_aco_norm0 = dw.prep_index_data(INDEX_DATA_FOLDER, normalize=True)

In [7]:
# Set time zones to local time
dw.update_time_zone(df_aco0, df_config)
dw.update_time_zone(df_aco_norm0, df_config)

,Date,Dataset,Sampling_Rate_kHz,FFT,Duration_sec,Thresholds_Hz,Filename,ZCR,MEANt,VARt,...,H_Renyi,H_pairedShannon,H_gamma,H_GiniSimpson,RAOQ,AGI,ROItotal,ROIcover,start_time,end_time
0,2/1/2019 1:59,"Gray's Reef, GA",48,512,300,1500;8000;24000,GRNMS_20190201_015958.wav,0.453619,-1.341507e-17,-0.000003,...,0.410787,0.092529,0.238378,0.016678,-0.032577,0.000833,0.002557,-0.005640,2019-01-31 20:59:00,2019-01-31 21:04:00
1,2/1/2019 2:04,"Gray's Reef, GA",48,512,300,1500;8000;24000,GRNMS_20190201_020458.wav,0.443633,5.796635e-16,-0.000004,...,0.406502,0.090813,0.231364,0.016554,-0.042187,0.000647,0.006517,-0.006124,2019-01-31 21:04:00,2019-01-31 21:09:00
2,2/1/2019 2:09,"Gray's Reef, GA",48,512,300,1500;8000;24000,GRNMS_20190201_020958.wav,0.468240,5.205142e-16,-0.000003,...,0.405054,0.089747,0.223626,0.016510,-0.041863,0.000868,0.002640,-0.005744,2019-01-31 21:09:00,2019-01-31 21:14:00
3,2/1/2019 2:14,"Gray's Reef, GA",48,512,300,1500;8000;24000,GRNMS_20190201_021458.wav,0.451701,-2.460612e-16,-0.000003,...,0.415808,0.095273,0.252608,0.016832,-0.019804,0.000784,0.007011,-0.005476,2019-01-31 21:14:00,2019-01-31 21:19:00
4,2/1/2019 2:19,"Gray's Reef, GA",48,512,300,1500;8000;24000,GRNMS_20190201_021958.wav,0.442630,1.949562e-16,-0.000003,...,0.426300,0.099364,0.269288,0.017116,0.016622,0.000863,0.005362,-0.005592,2019-01-31 21:19:00,2019-01-31 21:24:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241312,2/28/2019 23:47,ONC-MEF,16,512,150,1000;4000;8000,ONC_20190228_234730.wav,-0.023700,2.436953e-16,-0.000005,...,-0.295881,-0.153750,-0.327832,-0.046985,-0.244452,-0.005186,-0.000330,-0.006146,2019-02-28 15:47:00,2019-02-28 15:50:00
241313,2/28/2019 23:50,ONC-MEF,16,512,150,1000;4000;8000,ONC_20190228_235000.wav,-0.028639,-6.577406e-17,-0.000005,...,-0.243099,-0.129893,-0.297960,-0.035227,-0.220932,-0.005185,-0.000825,-0.006314,2019-02-28 15:50:00,2019-02-28 15:53:00
241314,2/28/2019 23:52,ONC-MEF,16,512,150,1000;4000;8000,ONC_20190228_235230.wav,-0.019951,-1.057590e-16,-0.000005,...,-0.225525,-0.121475,-0.284822,-0.031722,-0.208386,-0.005179,-0.000825,-0.006275,2019-02-28 15:52:00,2019-02-28 15:55:00
241315,2/28/2019 23:55,ONC-MEF,16,512,150,1000;4000;8000,ONC_20190228_235500.wav,-0.022851,4.400711e-16,-0.000005,...,-0.227100,-0.123574,-0.288507,-0.032252,-0.211902,-0.005187,-0.000330,-0.006266,2019-02-28 15:55:00,2019-02-28 15:58:00


## Add the annotation information to index dataframes

Add new columns to the index dataframes for all possible indices for both presence and count.

In [8]:
unique_codes = np.unique(df_codes['code']).tolist()

# Add new columns that are named using unique_fish_codes.
df_aco = dw.add_new_columns(df_aco0, unique_codes)
df_aco_norm = dw.add_new_columns(df_aco_norm0, unique_codes)

In [9]:
df_aco_anno = dw.add_annotations_to_df(df_aco, df_config, df_codes, annotations_folder)
df_aco_norm_anno = dw.add_annotations_to_df(df_aco_norm, df_config, df_codes, annotations_folder)

/Users/michelle/Documents/WORKSPACE/osa-mbon/python-prep/data_wrangler.py:292: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_new = pd.concat([df_new, df_sub], axis=0)
/Users/michelle/Documents/WORKSPACE/osa-mbon/python-prep/data_wrangler.py:292: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_new = pd.concat([df_new, df_sub], axis=0)


## Prep Seascaper Data


In [10]:
df_seascaper = dw.prep_seascaper_data(seascaper_folder, df_config)

## Create DuckDB

Save the main pandas dataframes to a duckdb file.

In [11]:
# Prep the dataframe list for export
df_dict = {
    "t_aco2": df_aco_anno,
    "t_aco_norm2": df_aco_norm_anno,
    "t_seascaper": df_seascaper,
    "t_fish_keywest": df_kw_anno,
    "t_fish_mayriver": df_mr_anno
}

dw.duckdb_export(db_file, df_dict)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [12]:
np.unique(df_aco_anno["Dataset"])

array(['Biscayne Bay, FL', 'Chukchi Sea, Hanna Shoal', "Gray's Reef, GA",
       'Key West, FL', 'May River, SC', 'ONC-MEF', 'OOI-HYDBBA106',
       'Olowalu (Maui, HI)'], dtype=object)

In [13]:
test = df_aco_anno[df_aco_anno["Dataset"] == "Key West, FL"]["Em_n"]
sum(test)

25906